AnimateDiff
===

**AnimateDiff-Lightning: Cross-Model Diffusion Distillation**

* Paper: https://arxiv.org/pdf/2403.12706

In [1]:
import torch
from diffusers import AnimateDiffPipeline, MotionAdapter, EulerDiscreteScheduler
from diffusers.utils import export_to_gif
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

device = "cuda"
dtype = torch.float16

step = 8  # Options: [1,2,4,8]
repo = "ByteDance/AnimateDiff-Lightning"
ckpt = f"animatediff_lightning_{step}step_diffusers.safetensors"
base = "emilianJR/epiCRealism"

adapter = MotionAdapter().to(device, dtype)
adapter.load_state_dict(load_file(
    hf_hub_download(repo ,ckpt), device=device)
)
pipe = AnimateDiffPipeline.from_pretrained(
    base, motion_adapter=adapter, torch_dtype=dtype
).to(device)
pipe.scheduler = EulerDiscreteScheduler.from_config(
    pipe.scheduler.config,
    timestep_spacing="trailing",
    beta_schedule="linear"
)

(…)ff_lightning_8step_diffusers.safetensors:   0%|          | 0.00/908M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/home/pyml/anaconda3/envs/py311/lib/python3.11/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [9]:
output = pipe(
    prompt="A dog running on the beach",
    guidance_scale=1.0,
    num_inference_steps=step
)
output

  0%|          | 0/8 [00:00<?, ?it/s]

AnimateDiffPipelineOutput(frames=[[<PIL.Image.Image image mode=RGB size=512x512 at 0x7F48305B0150>, <PIL.Image.Image image mode=RGB size=512x512 at 0x7F4830523A10>, <PIL.Image.Image image mode=RGB size=512x512 at 0x7F4830523890>, <PIL.Image.Image image mode=RGB size=512x512 at 0x7F4830594650>, <PIL.Image.Image image mode=RGB size=512x512 at 0x7F4830597590>, <PIL.Image.Image image mode=RGB size=512x512 at 0x7F4830596290>, <PIL.Image.Image image mode=RGB size=512x512 at 0x7F4830576AD0>, <PIL.Image.Image image mode=RGB size=512x512 at 0x7F4830575A50>, <PIL.Image.Image image mode=RGB size=512x512 at 0x7F4830575B90>, <PIL.Image.Image image mode=RGB size=512x512 at 0x7F4830456950>, <PIL.Image.Image image mode=RGB size=512x512 at 0x7F4830575F90>, <PIL.Image.Image image mode=RGB size=512x512 at 0x7F4830576650>, <PIL.Image.Image image mode=RGB size=512x512 at 0x7F4830576310>, <PIL.Image.Image image mode=RGB size=512x512 at 0x7F4830574450>, <PIL.Image.Image image mode=RGB size=512x512 at 0x7F483

In [10]:
export_to_gif(output.frames[0], "animation.gif")

'animation.gif'

![](animation.gif)